In [1]:
import Data_Preprocessing.Graph_Data as gd
from Data_Preprocessing.Graph_Data import Molecule_data
from models.AttentiveFPModel import AttentiveFP
import matplotlib.pyplot as plt
import torch
from torch_geometric.loader import DataLoader
import os

# Check Cuda

In [2]:
if torch.cuda.is_available():  
    device = "cuda:1"
    print("cuda:1")
else:  
    device = "cpu" 
    print(torch.cuda.is_available())

cuda:1


# Import Model

In [3]:
model = AttentiveFP(in_channels=114, hidden_channels=292, out_channels=1,
                    num_layers=3, num_timesteps=1,
                    dropout= 0.05115077176824934).to(device)

In [4]:
model_file_name = 'Graph_attention_best_12.model'
model.load_state_dict(torch.load(model_file_name))

<All keys matched successfully>

# BandGap Predication

In [5]:
def smilePredication(smile):
    smile_graph = {}
    band_gap_arr = []
    
    g = gd.smile_to_graph(smile)
    if g != None:
        smile_graph[smile] = g
        band_gap_arr.append(0.00)
        smiles_array.append(smile)
        #Check the file if exists then delete 
        if os.path.exists("data/processed/predicate_data_set.pt"):
            os.remove("data/processed/predicate_data_set.pt")
            
        pred_data = Molecule_data(root='data', dataset='predicate_data_set',
                                 y=band_gap_arr,smile_graph=smile_graph,smiles=smiles_array)
        TRAIN_BATCH_SIZE = 17
        pred_loder   = DataLoader(pred_data,batch_size=TRAIN_BATCH_SIZE,shuffle=True)
        
        for data in pred_loder:
            data = data.to(device)
            return model(data.x.float(), data.edge_index,data.batch)
    else: 
        print("Error while converting smile into graph")
        return


In [12]:
band_gap = smilePredication('C1(=O)N(C(CN1Cc1ccccc1)COC(=O)C)S(=O)(=O)c1ccc(cc1)C')

Pre-processed data data/processed/predicate_data_set.pt not found, doing pre-processing...
Converting SMILES to graph: 1/1
Graph construction done. Saving to file.


In [23]:
print('Predicated BandGap: ',band_gap)

Predicated BandGap:  tensor([[3.6471]], device='cuda:1', grad_fn=<AddmmBackward>)
